In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.special import softmax
import pandas as pd
import tqdm
import os
import imageio
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import BPHelp
from BPHelp import *
from scipy.optimize import minimize 
import pickle
from connectivity import *

import plots
from importlib import reload

pd.set_option('display.max_rows', 1000)
np.random.seed(seed=233423)

### lockdown conditions

In [3]:
# global variables for lockdown conditions...

# SI mean and std-dev used for (rough) estimator for R0 given case records in a given community
serial_interval_mean = 5.4
serial_interval_sd = 4.5

# how far back to look (in days) when estimating the current R0 for a given community?
lookback_time = 8

# only check the lockdown condition when there are at least this many cases
case_threshold = 5

# threshold for the estimated R0
R0_threshold = 1.0 # nominal R0 value for triggering lockdown
R0_sigma_threshold = 1.0 # number of sigmas the estimated R0 must be above R0_threshold to actually trigger a lockdown

# function takes in case counts n, observation times t (in days), also serial intervals mean and st and initial guess for R0
# returns rough maximum likelihood for R0 and an approximate (1-sigma) uncertainty
def estimate_R0(n, t, serial_interval_mean, serial_interval_sd, R0_initial, tol=0.05):
    
    # objective function fo minimize to get MLE of exponential rate, r
    def f(r, n, t):

        sn = np.sum(n)
        snt = np.sum(n*t)
        sert = np.sum(np.exp(r*t))
        return - ( sn*(np.log(sn) - np.log(sert)) + r*snt - sn )
    
    # initial estimate of r given R0 etc
    a = serial_interval_mean**2/serial_interval_sd**2
    b = serial_interval_mean/serial_interval_sd
    r0 = b*(R0_initial**(1./a) - 1)
    
    # exponential rate maximum likelihood estimator
    r = minimize(fun = lambda x: f(x, n, t), x0=r0, tol=tol).x[0]
    
    # extract exponential normalization
    A = np.sum(n)/np.sum(np.exp(r*t))
    
    # convert back to R0
    R0 = (1 + r/b)**a
    
    # calculate rough error bars
    F_rr = A*np.sum(t**2*np.exp(r*t))
    F_rA = np.sum(t*np.exp(r*t))
    F_AA = np.sum(n)/A**2
    C = np.linalg.inv( np.array([[F_rr, F_rA],[F_rA, F_AA]]) )
    sigma_r = np.sqrt(C[0,0])
    sigma_A = np.sqrt(C[1,1])
    
    sigma_R0 = sigma_r*(a/b)*(1+r/b)**(a-1)
    
    return R0, sigma_R0

# benchmark lockdown condition
def lockdown_condition_benchmark(community_confirmed_cases, c):

    # estimate R0 and it's uncertainty
    R0_, sigma_R0_ = estimate_R0(community_confirmed_cases[-lookback_time:, c], np.arange(lookback_time), serial_interval_mean, serial_interval_sd, community_R0[-1, c], tol=0.05)

    # if one sigma detection of R0 > 1, lockdown=True, else lockdown=False 
    if R0_ - (R0_sigma_threshold/2)*sigma_R0_ >= 1:
        return True, R0_
    else:
        return False, R0_

### set-up parameters

In [4]:
# run epochs until either the number of active infections is > max_infections or we have past max_time (simulation days)
max_infections = 10000 # maximum number of active cases to simulate
max_time = 90 # maximum number of days to simulate

# R0, its dispersion, and corresponding negative binomial distribution parameters
R0 = 2.5
R0_dispersion = 0.16
nbinom_p = 1. - R0_dispersion
nbinom_n = R0*nbinom_p/(1.-nbinom_p)

# contact tracing success rate, quarantine efficacy, isolation efficacy
p_traceable = 0.8 # ie., percentage of population who are active contact tracing app users
quarantine_efficacy = 0.0
isolation_efficacy = 1.0

# lockdown period and efficacy
lockdown_period = 14 # days
lockdown_efficacy = 0.8
lockdown_condition = lockdown_condition_benchmark

# probability that someone will be asymptomatic and their relative infectiousness
p_asymptomatic = 0.1
asymptomatic_infectiousness = 0.1

# lognormal for incubation period from Bi+2020
incubation_scale = np.exp(1.57)
incubation_s = 0.65
incubation_distribution = stats.lognorm(scale=incubation_scale, s=incubation_s)

# serial interval distribution: skew normal (from Hellewell+2020)
serial_k = 0.7 # skewness: 0.7 corresponds to 30% of infections during pre-symptomatic phase
serial_sd = 2 # std-deviation of the skew normal

# delay from symptom onset to covid-specific symptom onset worthy of reporting/self-isolating
# time to "first consultation" from SK Sgpor. HK mu=2.1, sd=2.65
# alternative: time to onset of first symptoms mu=1, sd=2 (ish)
symptom_mean = 2.1
symptom_sd = 2.65
symptom_a = symptom_sd**2/symptom_mean
symptom_scale = symptom_mean/symptom_a
isolation_delay_distribution = stats.gamma(scale=symptom_scale, a=symptom_a)

# recovery distribution for onset-recovery time
recovery_a = (1/0.35)**2
recovery_scale = 24.7/recovery_a
onset_to_recovery_distribution = stats.gamma(scale=recovery_scale, a=recovery_a)

# delay to getting a test confirmation
testing_delay = 1.0

# number of initial cases
n_initial_cases = 20

# initial infection time distribution
inital_infection_time_distribution = lambda n_initial_cases:np.random.uniform(0, 3, n_initial_cases)

# network model for the UK (including movements for workplace, school and leisure activities)
connectivity_class = ThreePartConnectivity('../connectivity_data_UK/three_part_connectivity_model.pkl')

# number of simulations to run
num_sims = 10

# lockdown condition
lockdown_condition = lockdown_condition_benchmark

### run simulations

In [6]:
# holds trajectories for each simulation (ie., keeps track of the community level number of infections, lockdowns, etc)
trajectories = []

# loop over how many simulations you want ot run in your ensemble
for s in range(num_sims):
    
    print('simulation #{}...'.format(s))
    
    # ------------- INITIAL INFECTIONS ------------- #
    
    # create data frame to hold the infection tree
    dataframe = pd.DataFrame(columns=['infection time', # when did they get infected
                                      'incubation period', # incubation period
                                      'symptom onset time', # when did their symptomns onset?
                                      'isolation time', # at what time did they isolate?
                                      'case confirmation time', # at what time was the case confirmed (via a test)?
                                      'quarantine time', # at what time did they quarantine?
                                      'recovery time', # at what time did they recover?
                                      'asymptomatic', # are they asymptomatic? (True/False)
                                      'parent id', # id of the node that infected them
                                      'id', # node id
                                      'community', # community where they live
                                      'community of parent', # community where the person who infected them lives
                                      'parent infection time', # when did the person who infected them get infected?
                                      'reproduction number', # how many people did they go on to infect
                                      'traceable', # are they traceable, in princple, via contact tracing? (ie., do they have the app)
                                      'logged', # did they log their case in the contact tracing app? (ie., are they both traceable and not asymptomatic)
                                      'contact traced', # did they get successfully traced via contact tracing?
                                      'tip node']) # are they a tip node? (ie., have they had a chance to infecton others yet? set to True when the node is created, and flipped ot False once they have generated their secondary cases)
    
    # number of comunities
    n_communities = connectivity_class.n_communities

    # index to keep track of the current "highest id" of infected people in the tree
    id_index = 0

    # time of infection = 0 for all parent nodes
    infection_time = inital_infection_time_distribution(n_initial_cases)

    # draw incubation times, asymptomitic bools, and delay from onset of symptoms to isolation, and recovery times
    incubation_period = incubation_distribution.rvs(size=n_initial_cases)
    asymptomatic = stats.bernoulli.rvs(p_asymptomatic, size=n_initial_cases)
    isolation_delay = isolation_delay_distribution.rvs(size=n_initial_cases)
    recovery_time = infection_time + incubation_period + isolation_delay + np.maximum(0, onset_to_recovery_distribution.rvs(size=n_initial_cases) - symptom_mean)

    # draw whether parent nodes are traceable, and traced
    traceable = stats.bernoulli.rvs(p=p_traceable, size=n_initial_cases)
    logged = traceable * (1 - asymptomatic)
    contact_traced = np.zeros(n_initial_cases)

    # generate ids of these parent cases and update the id index
    new_ids = np.arange(id_index, id_index + n_initial_cases)
    id_index = max(new_ids) + 1

    # which communities are the initial infected people in? assign initial communities randomly, weighted by population per community
    communities = np.random.choice(np.arange(n_communities), p=np.ones(n_communities)/n_communities, size=n_initial_cases)

    # add the parent nodes to the dataframe
    for i in range(n_initial_cases):
        dataframe = dataframe.append({'infection time':infection_time[i],
                      'incubation period':incubation_period[i],
                      'symptom onset time':infection_time[i] + incubation_period[i],
                      'isolation time':infection_time[i] + incubation_period[i] + isolation_delay[i],
                      'case confirmation time':infection_time[i] + incubation_period[i] + isolation_delay[i] + testing_delay,
                      'quarantine time':infection_time[i] + incubation_period[i] + isolation_delay[i], # set equal to isolation time for initial nodes
                      'recovery time':recovery_time[i],
                      'asymptomatic':asymptomatic[i],
                      'parent id':None,
                      'id':new_ids[i],
                      'community':communities[i],
                      'community of parent':None,
                      'parent infection time':None,
                      'traceable':traceable[i],
                      'logged':logged[i],
                      'contact traced':contact_traced[i],
                      'reproduction number':None, # they have not infected anyone yet; these will get updated as we run the process forward
                      'tip node':True}, # flag for whether the node is a tip or not
                      ignore_index=True)

    # initialize the tip nodes (to begin with, all of the nodes are tip nodes)
    # a tip node is a node in the branching tree that is on one of the "tips" of
    # the branches, ie., they are infected and have not had a chance to infect others
    # yet.
    tipnodes = dataframe[dataframe['tip node'] == True]
    n_tip_nodes = tipnodes.id.count()


    # ------------- RECORDS ------------- #
    
    # arrays to keep track of number of active and symptomatic infections as a function of time (we will append to these as the tree grows)
    time = [0] # current time up-to-which the branching tree is complete (ie., no new infections can now occur before this time), initialized to t=0
    active_infections = np.array([n_initial_cases]) # number of people infected (not yet recovered)
    free_infections = np.array([n_initial_cases]) # number of people infected (not yet recovered) and not isolated
    community_active_infections = np.array([[sum(communities == i) for i in range(n_communities)]]) # active infections per community
    community_confirmed_cases = np.array([[0 for i in range(n_communities)]]) # number of confirmed cases per community (initialized to zero)

    # lockdown status for each commuity and the planned time for unlocking each community (=0 if not currently locked down)
    lockdown = [False]*n_communities # all communities begin not locked down (lockdown = False)
    unlock_time = np.array([0]*n_communities) # planned time for unlocking community = 0 for all communities initially
    lockdown_history = np.array([lockdown]) # array to keep history whether each community was locked down or not at any given time

    # initial R0 estimates for each community (initialized to zeros)
    community_R0 = np.array([[0 for i in range(n_communities)]])


    # ------------- MAIN LOOP ------------- #
    
    # progress bar
    pbar = tqdm.tqdm_notebook(total=max_time, desc='Time')

    # three conditions tested to continue simulating: is the number of active infections less than the max
    # infections threshold? Is the current time up-to-which the branching tree is complete less than the max
    # time threshold? and, are there any "tip nodes" in the tree, ie., infected people who have not had a
    # chance to infect others yet? (when there are no tip nodes, the branching process has terminated since
    # there are no possibilities for further new infections).
    while (active_infections[-1] < max_infections) and (time[-1] < max_time) and n_tip_nodes > 0:
        
        # ------------- DO INFECTIONS FOR THIS PARENT NODE ------------- #

        # set the current parent node as the "most recently infected tip node"
        parent_id = tipnodes[tipnodes['infection time'].values == min(tipnodes['infection time'].values)].id.values[0]

        # time of infection, incubation period and time of isolation of the parent node
        parent_infection_time = dataframe[dataframe.id == parent_id]['infection time'].values[0]
        parent_incubation_period = dataframe[dataframe.id == parent_id]['incubation period'].values[0]
        parent_isolation_time = dataframe[dataframe.id == parent_id]['isolation time'].values[0]
        community_of_parent = dataframe[dataframe.id == parent_id]['community'].values[0]
        parent_logged = dataframe[dataframe.id == parent_id]['logged'].values[0]
        parent_asymptomatic = dataframe[dataframe.id == parent_id]['asymptomatic'].values[0]
        parent_quarantine_time = dataframe[dataframe.id == parent_id]['quarantine time'].values[0]
        parent_recovery_time = dataframe[dataframe.id == parent_id]['recovery time'].values[0]
        parent_case_confirmation_time = dataframe[dataframe.id == parent_id]['case confirmation time'].values[0]

        # draw number of new people this parent node will infect
        n_new_infections = stats.nbinom.rvs(n=nbinom_n*(1 - parent_asymptomatic*(1-asymptomatic_infectiousness)), p=nbinom_p)
        child_ids = np.arange(id_index, id_index + n_new_infections)

        # serial intervals for each new infection
        serial_intervals = np.abs(stats.skewnorm.rvs(loc=parent_incubation_period, scale=serial_sd, a=serial_k, size = n_new_infections))

        # infection times for the child nodes
        infection_time = parent_infection_time + serial_intervals

        # draw incubation times for new infections
        incubation_period = incubation_distribution.rvs(size=n_new_infections)
        symptom_onset_time = infection_time + incubation_period

        # symptomatic or not for new infections
        asymptomatic = stats.bernoulli.rvs(p_asymptomatic, size=n_new_infections)

        # traceable or not for new infections? and, are they traced? (latter requires they will be both symptomatic and are traceable)
        traceable = stats.bernoulli.rvs(p_traceable, size=n_new_infections)
        logged = (1 - asymptomatic) * traceable
        contact_traced = traceable * parent_logged

        # delay between symptom onset and isolation (ie., having symptoms substantial enough to want to isolate and report)
        isolation_delay = isolation_delay_distribution.rvs(size=n_new_infections)

        # recovery time (infection time + onset + onset->recovery time)
        recovery_time = infection_time + incubation_period + isolation_delay + np.maximum(0, stats.gamma.rvs(scale=recovery_scale, a=recovery_a, size=n_new_infections) - symptom_mean)

        # isolation time: we are assuming they will self-isolate with a delay given above (ie, at onset of symptoms worthy of reporting+isolation)
        # if they were symptomatic, then if they were contact traced they get isolated at their symptom onset time or, if it comes later, the time that their parent's case was confirmed
        # if they were symptomatic but not contact traced, they get isolated with the usual delay
        # if they were asymptomatic, they never get isolated so set isolation time == recovery time
        isolation_time = (1-asymptomatic) * (contact_traced * np.maximum(symptom_onset_time, parent_case_confirmation_time) + (1-contact_traced) * (symptom_onset_time + isolation_delay + testing_delay) ) + asymptomatic * recovery_time

        # trim the branches given possible quarantine, isolation, or lockdown of parent node...
        # some of the infections generated above may have been generated after the parent node was actually
        # already isolated, quarantines, or had lockdown imposed on them. Therefore, we take the cases generated
        # above and "prune" them subject to when the parent node was quarantined/isolated/locked down

        # probability of each infection being realized depends on whether the parent was free, quarantined, or isolated
        # if infection occured before parent was quarantined, accepted for certain
        # if infection occured while parent was quarantined (not isolated), accept with p=quarantine_efficacy
        # if infection occured after parent was isolated but before they were recovered, accept with p=isolation_efficacy
        p_accepted = ( 1.*(infection_time < parent_quarantine_time) + (1.-quarantine_efficacy)*(infection_time > parent_quarantine_time)*(infection_time < parent_isolation_time) + (1.-isolation_efficacy)*(infection_time > parent_isolation_time) ) * (infection_time < parent_recovery_time)

        # probability of each infection being realized also depends on whether the parent node was under lockdown at the time
        # if an infection occured when the parent node was locked down, if get accepted with probability (1. - lockdown_efficacy)
        p_accepted = p_accepted * (1 - lockdown_efficacy*(infection_time < unlock_time[community_of_parent]))

        # select which infections make the cut, given whether parent was free, quarantined, isolated, or under lockdown at the suggested infection time
        realized_infections = np.atleast_1d(stats.bernoulli.rvs(p_accepted)==1) # draw True/False for each infection with the accumulated acceptance probability

        # are there any children nodes left after the above cuts? if so, add them to the tree!
        if sum(realized_infections) > 0:

            # trim the branches based on the above cuts
            infection_time = infection_time[realized_infections]
            incubation_period = incubation_period[realized_infections]
            symptom_onset_time = symptom_onset_time[realized_infections]
            isolation_time = isolation_time[realized_infections]
            recovery_time = recovery_time[realized_infections]
            traceable = traceable[realized_infections]
            logged = logged[realized_infections]
            contact_traced = contact_traced[realized_infections]
            asymptomatic = asymptomatic[realized_infections]

            # number of actual new infections
            n_new_infections = sum(realized_infections)
            child_ids = np.arange(id_index, id_index + n_new_infections)
            id_index = max(child_ids) + 1

            # set the reproduction number of the parent node now that their isecondary nfections have been realized
            dataframe.at[dataframe[dataframe['id'] == parent_id].index, 'reproduction number'] = n_new_infections

            # quarantined? if parent was symptomatic and both parent and child are contact traced, then child gets quarantined
            # if they are quarantined, they get quarantined at the time that their parent's case was confirmed
            # if this time was after the time they were going to isolate anyway (due to their own symptoms) then we set quaraantine time equal to their isolation time
            # if child was not quarantined, quarantine time also set as identical to their isolation time
            # NB if quarantine_efficacy = 0 then quarantining has no effect anyway
            quarantined = contact_traced
            quarantine_time = quarantined * np.minimum(parent_case_confirmation_time, isolation_time) + (1. - quarantined) * isolation_time

            # which communitites do the children live in?
            communities = connectivity_class.draw_locations_of_children(c=community_of_parent, n=n_new_infections)

            # add each new case to the dataframe
            for i in range(n_new_infections):
                dataframe = dataframe.append({'infection time':infection_time[i],
                      'incubation period':incubation_period[i],
                      'symptom onset time':symptom_onset_time[i],
                      'isolation time':isolation_time[i],
                      'quarantine time':quarantine_time[i],
                      'case confirmation time':isolation_time[i] + testing_delay,
                      'recovery time':recovery_time[i],
                      'asymptomatic':asymptomatic[i],
                      'parent id':parent_id,
                      'id':child_ids[i],
                      'community':communities[i],
                      'community of parent':community_of_parent,
                      'parent infection time':parent_infection_time,
                      'traceable':traceable[i],
                      'logged':logged[i],
                      'contact traced':contact_traced[i],
                      'reproduction number':None,
                      'tip node':True}, ignore_index=True)

        # if no children made the cut, set reproduction number of this parent node to zero and move on to the next tip node
        else:
            dataframe.at[dataframe[dataframe['id'] == parent_id].index, 'reproduction number'] = 0

        # set the current parent node to no longer be a tip node (it can no longer infect anyone new)
        dataframe.at[dataframe[dataframe['id'] == parent_id].index, 'tip node'] = False
        
        
        # ------------- ANALYSIS OF TREE UP TO THIS POINT AND IMPOSING OF LOCKDOWN CONDITIONS ------------- #

        # analysis of the tree up to the current "completeness time": this is the time before which nothing new can now 
        # happen. The completeness time is the infection time of the parent node that we just spawned children for - 
        # nothing can any longer happen before this point (since they were the youngest parent node in the tree, and they
        # have now done all their infecting)

        # current "completeness time" of the tree
        completeness_time = parent_infection_time
        completeness_day = np.floor(completeness_time) # the day up to which we are now complete (ie, floor to the nearest integer)

        # has our completeness time moved forward by at least one day? if so, calculate number of active cases etc for the new days that we now have a complete tree for
        previous_completeness_day = time[-1]
        if completeness_day > previous_completeness_day:

            # calculate the number of active and free infections and number of confirmed cases, as a function of time
            for t in np.arange(previous_completeness_day + 1, completeness_day + 1):

                # extract dataframes for active infections, free infections and confirmed cases
                active_infection_dataframe = dataframe[(dataframe['infection time'] < t) & (dataframe['recovery time'] > t)]
                free_infection_dataframe = dataframe[(dataframe['infection time'] < t) & (dataframe['isolation time'] > t)]
                confirmed_case_dataframe = dataframe[(dataframe['asymptomatic'] == 0) & (dataframe['case confirmation time'] > t-1) & (dataframe['case confirmation time'] <= t)]

                # total number of active and free (not-isolated) infections (unobservable)
                active_infections = np.append(active_infections, active_infection_dataframe.id.count())
                free_infections = np.append(free_infections, free_infection_dataframe.id.count())

                # active infections in each community
                community_active_infections = np.append(community_active_infections,
                                                        current_community_infections(active_infection_dataframe, n_communities),
                                                        axis=0)
                
                # number of symptomatic infections reported on day t per community
                community_confirmed_cases = np.append(community_confirmed_cases,
                                                      current_community_infections(confirmed_case_dataframe,n_communities),
                                                      axis=0)          

                # append to the time array
                time.append(t)

                # ------------- LOCKDOWN CONDITION AND IMPLEMENTATION ------------- #

                # unlock any communities that have been locked down for > lockdown_period
                for c in range(n_communities):
                    if lockdown[c] is True and time[-1] >= unlock_time[c]:
                        print("unlocking community number {}".format(c))
                        unlock_time[c] = 0
                        lockdown[c] = False

                # estimated R0 per community, initialize new lockdowns where appropriate
                estimated_R0 = [0 for i in range(n_communities)]
                for c in range(n_communities):

                    # lockdown condition newly satisfied?
                    if time[-1] > lookback_time and lockdown[c] is False and np.sum(community_confirmed_cases[-lookback_time:, c]) > case_threshold:

                        # test lockdown condition
                        condition, R0_ = lockdown_condition(community_active_infections, c)
                        estimated_R0[c] = R0_
                        
                        # if lockdown condition is staisfied, initialize lockdown
                        if condition is True:

                            # lockdown this community for the speficied lockdown period
                            print("locking down community number {}".format(c))
                            lockdown[c] = True
                            unlock_time[c] = time[-1] + lockdown_period # set the time to unlock this community

                            # prune the tree now that this community has been locked down:
                            # any infections that were generated "in the future" relative to 
                            # the initiation of this new lockdown are now less likely ot have 
                            # actually occurred. We therefore take those nodes in the tree and
                            # remove them with probability lockdown_efficacy

                            # ids of the tip nodes that were actually infected during the newly imposed lockdown
                            lockdown_indexs = dataframe[(dataframe['community of parent'] == c) & (dataframe['infection time'] > completeness_time) & (dataframe['infection time'] < completeness_time + lockdown_period)].index
                            if len(lockdown_indexs) > 0:
                                # ids for which tip nodes to delete? drop the lockdown_id nodes out with probability p=lockdown_efficacy
                                delete_indexs = lockdown_indexs[np.atleast_1d(stats.bernoulli.rvs(p=lockdown_efficacy, size=len(lockdown_indexs))==1)]

                                # if any are to be dropped, delete them from the dataframe
                                if len(delete_indexs) > 0:
                                    dataframe = dataframe.drop(delete_indexs)

                # record the estimated R0 for each community on this day
                community_R0 = np.append(community_R0, np.array([estimated_R0]), axis=0)

                # update lockdown history
                lockdown_history = np.append(lockdown_history, np.array([lockdown]), axis=0)

        # update the tip nodes and count how many there are
        tipnodes = dataframe[dataframe['tip node'] == True]
        n_tip_nodes = tipnodes.id.count()

        # calculate the effective reproduction number for the whole country
        internal_nodes = dataframe[dataframe['tip node'] == False] # these are the nodes of the branching tree that have already done all their infecting (ie,. they are not tip nodes)
        R_eff = np.mean(internal_nodes['reproduction number'].values) # mean reproduction number for all nodes

        # update progress bar
        pbar.n = round(completeness_time, 2)
        pbar.set_postfix(ordered_dict={'active infections':active_infections[-1], 'free infections':free_infections[-1], 'tip nodes':n_tip_nodes, 'R_eff':R_eff}, refresh=True)
        pbar.refresh()
        
    # update the trajectory for this sim
    trajectory = {'time':time,
              'active_infections':active_infections,
              'free_infections':free_infections,
              'commumintiy_active_infections':community_active_infections,
              'community_confirmed_cases':community_confirmed_cases,
              'lockdown_history':lockdown_history,
              'R_eff':R_eff}
    trajectories.append(trajectory)
    
# save the trajectories to file
with open('test_results/simulations_lockdown_{}sims'.format(num_sims), "wb") as file:
    pickle.dump(trajectories, file)

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:97: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


locking down community number 6202
